In [17]:
# ============================================================
# 0. Imports & Config
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

from xgboost import XGBClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Folder & filenames — update these to match your setup
DATA_DIR = "/kaggle/input/tcga-data/data/processed"   # folder where aligned files live

RNA_FILE   = "rnaseq_aligned.csv"          # rows = samples, columns = genes
METH_FILE  = "methylation_aligned.csv"  # rows = samples, columns = CpGs
CLIN_FILE  = "clinical_aligned.csv"     # rows = samples, columns incl. stage

Using device: cpu


In [18]:
# ============================================================
# 1. Load Data
# ============================================================
rna   = pd.read_csv(os.path.join(DATA_DIR, RNA_FILE), index_col=0)
meth  = pd.read_csv(os.path.join(DATA_DIR, METH_FILE), index_col=0)
clin  = pd.read_csv(os.path.join(DATA_DIR, CLIN_FILE), index_col=0)

print("RNA shape:        ", rna.shape)
print("Methylation shape:", meth.shape)
print("Clinical shape:   ", clin.shape)

RNA shape:         (1092, 60660)
Methylation shape: (1092, 14165)
Clinical shape:    (1092, 7)


In [19]:
# ============================================================
# 2. Build Tumor Stage Labels (Normalized to I / II / III / IV)
# ============================================================

def normalize_stage(s):
    """Map messy stage strings to I, II, III, IV; else NaN."""
    if pd.isna(s):
        return np.nan
    s = str(s).lower()
    # Order matters: check 'iv' before 'iii', etc.
    if "iv" in s:
        return "IV"
    if "iii" in s:
        return "III"
    if "ii" in s:
        return "II"
    if "i" in s:
        return "I"
    return np.nan

if "stage_filled" in clin.columns:
    raw_stage = clin["stage_filled"]
elif "stage_best" in clin.columns:
    raw_stage = clin["stage_best"]
else:
    raise ValueError("No 'stage_filled' or 'stage_best' in clinical_aligned.csv")

labels = raw_stage.apply(normalize_stage)

print("Stage distribution BEFORE dropping NaNs:")
print(labels.value_counts(dropna=False))

Stage distribution BEFORE dropping NaNs:
stage_best
II     620
III    249
I      179
NaN     24
IV      20
Name: count, dtype: int64


In [20]:
# ============================================================
# 3. Align Samples Across RNA, Methylation, Labels
# ============================================================

common_samples = set(rna.index) & set(meth.index) & set(labels.index)
common_samples = sorted(common_samples)

rna   = rna.loc[common_samples]
meth  = meth.loc[common_samples]
labels = labels.loc[common_samples]

# Drop samples with unknown stage after normalization
mask = labels.notna()
rna   = rna.loc[mask]
meth  = meth.loc[mask]
labels = labels.loc[mask]

print("\nAfter alignment & dropping NaN stages:")
print("RNA:", rna.shape, "Methylation:", meth.shape, "Labels:", labels.shape)
print(labels.value_counts())


After alignment & dropping NaN stages:
RNA: (1068, 60660) Methylation: (1068, 14165) Labels: (1068,)
stage_best
II     620
III    249
I      179
IV      20
Name: count, dtype: int64


In [21]:
# ============================================================
# 4. Preprocess: Impute & Normalize BEFORE PCA
# ============================================================

# --- RNA ---
# Impute per-feature (gene) median, then standardize across samples
rna_imputed = rna.copy()
rna_imputed = rna_imputed.apply(lambda col: col.fillna(col.median()), axis=0)

# --- Methylation ---
meth_imputed = meth.copy()
meth_imputed = meth_imputed.apply(lambda col: col.fillna(col.median()), axis=0)


def compute_pca_from_samples_df(df, n_components=20):
    """
    df: samples × features
    1) Standardize features
    2) PCA
    Returns: (samples × n_components)
    """
    X = df.values  # samples × features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    n_comp = min(n_components, X_scaled.shape[0])  # cannot exceed #samples
    pca = PCA(n_components=n_comp, random_state=SEED)
    X_pca = pca.fit_transform(X_scaled)

    print(f"PCA input {df.shape} -> {X_pca.shape}, "
          f"explained variance={pca.explained_variance_ratio_.sum():.2%}")
    return X_pca

print("\n--- Running PCA ---")
rna_pca  = compute_pca_from_samples_df(rna_imputed,  n_components=20)
meth_pca = compute_pca_from_samples_df(meth_imputed, n_components=20)

# Multimodal PCA fusion
multi_pca = np.concatenate([rna_pca, meth_pca], axis=1)
print("Multimodal PCA shape:", multi_pca.shape)

# Ensure no NaNs
rna_pca   = np.nan_to_num(rna_pca)
meth_pca  = np.nan_to_num(meth_pca)
multi_pca = np.nan_to_num(multi_pca)


--- Running PCA ---
PCA input (1068, 60660) -> (1068, 20), explained variance=35.69%
PCA input (1068, 14165) -> (1068, 20), explained variance=51.42%
Multimodal PCA shape: (1068, 40)


In [22]:
# ============================================================
# 5. Define a Stable VAE for Multimodal PCA
# ============================================================

class VAE(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int = 4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        self.mu = nn.Linear(64, latent_dim)
        self.logvar = nn.Linear(64, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
        )

    def encode(self, x):
        h = self.encoder(x)
        mu = self.mu(h)
        logvar = self.logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z  = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar, z


def vae_loss(recon_x, x, mu, logvar, beta: float = 1.0):
    recon_loss = nn.MSELoss()(recon_x, x)
    kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + beta * kld

In [23]:
# ============================================================
# 6. Train VAE on Normalized Multimodal PCA
# ============================================================

# Further standardize multi_pca before feeding to VAE
scaler_vae = StandardScaler()
multi_pca_scaled = scaler_vae.fit_transform(multi_pca)
multi_pca_scaled = np.nan_to_num(multi_pca_scaled)

X_tensor = torch.tensor(multi_pca_scaled, dtype=torch.float32).to(device)
dataset = TensorDataset(X_tensor)
loader  = DataLoader(dataset, batch_size=16, shuffle=True)

vae = VAE(input_dim=multi_pca_scaled.shape[1], latent_dim=4).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=5e-4)

n_epochs = 50
vae.train()
for epoch in range(1, n_epochs + 1):
    epoch_loss = 0.0
    for (batch,) in loader:
        batch = batch.to(device)
        recon, mu, logvar, z = vae(batch)
        loss = vae_loss(recon, batch, mu, logvar, beta=1.0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch.size(0)

    epoch_loss /= len(dataset)
    if epoch % 10 == 0 or epoch == 1:
        print(f"VAE Epoch {epoch:3d}/{n_epochs}  Loss: {epoch_loss:.4f}")

# Extract stable latent space (mu)
vae.eval()
with torch.no_grad():
    mu, logvar = vae.encode(X_tensor)
latent = mu.cpu().numpy()
latent = np.nan_to_num(latent)
print("VAE latent shape:", latent.shape)

VAE Epoch   1/50  Loss: 1.0080
VAE Epoch  10/50  Loss: 1.0012
VAE Epoch  20/50  Loss: 1.0008
VAE Epoch  30/50  Loss: 1.0008
VAE Epoch  40/50  Loss: 1.0006
VAE Epoch  50/50  Loss: 1.0004
VAE latent shape: (1068, 4)


In [24]:
# ============================================================
# 7. Helper: XGBoost Evaluation Function
# ============================================================

def evaluate_xgb(X, labels, name):
    """
    X: np.array, samples × features
    labels: pd.Series of normalized stages ("I", "II", "III", "IV")
    """
    mask = labels.notna()
    X = X[mask]
    y = labels[mask]

    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc,
        test_size=0.2,
        random_state=SEED,
        stratify=y_enc
    )

    clf = XGBClassifier(
        n_estimators=400,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=2.0,
        objective="multi:softprob",
        eval_metric="mlogloss",
        tree_method="hist",
        random_state=SEED,
        use_label_encoder=False
    )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"\n=== {name} ===")
    print("Weighted F1:", round(f1, 4))

    # To avoid mismatch: only report on labels actually in y_test
    test_classes = np.unique(y_test)
    print(
        classification_report(
            y_test,
            y_pred,
            labels=test_classes,
            target_names=le.inverse_transform(test_classes),
            zero_division=0
        )
    )

    return clf, f1

In [25]:
# ============================================================
# 8. Run XGBoost on Different Representations
# ============================================================

models = {}
scores = {}

models["XGB_RNA"],   scores["RNA PCA"]        = evaluate_xgb(rna_pca,   labels, "RNA PCA (XGB)")
models["XGB_METH"],  scores["Methyl PCA"]     = evaluate_xgb(meth_pca,  labels, "Methylation PCA (XGB)")
models["XGB_MULTI"], scores["Multimodal PCA"] = evaluate_xgb(multi_pca, labels, "Multimodal PCA (XGB)")
models["XGB_VAE"],   scores["VAE latent"]     = evaluate_xgb(latent,    labels, "VAE latent (XGB)")

print("\n===== Summary of Weighted F1 Scores (XGBoost) =====")
for k, v in scores.items():
    print(f"{k:20s}: {v:.4f}")


=== RNA PCA (XGB) ===
Weighted F1: 0.4663
              precision    recall  f1-score   support

           I       0.09      0.03      0.04        36
          II       0.60      0.84      0.70       124
         III       0.29      0.18      0.22        50
          IV       0.00      0.00      0.00         4

    accuracy                           0.53       214
   macro avg       0.25      0.26      0.24       214
weighted avg       0.43      0.53      0.47       214


=== Methylation PCA (XGB) ===
Weighted F1: 0.4762
              precision    recall  f1-score   support

           I       0.29      0.11      0.16        36
          II       0.59      0.80      0.68       124
         III       0.32      0.20      0.25        50
          IV       0.00      0.00      0.00         4

    accuracy                           0.53       214
   macro avg       0.30      0.28      0.27       214
weighted avg       0.46      0.53      0.48       214


=== Multimodal PCA (XGB) ===
Weight